In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
import pandas as pd

In [8]:
def scrape(url,df_dict):  #to scrape 'Title','Company','Location','Salary' and 'JobSummary' from each HTML page
    # establishing the connection to the webpage
    response = requests.get(url)

    # You can use status codes to understand how the target server responds to your request.
    #Ex. 200 = OK, 400 = Bad Request, 403 = Forbidden, 404 = Not Found
    print('Status Code: ',response.status_code)

    if response.status_code==200:
        # Pull HTML string out of requests and convert to a python string
        html = response.text
        soup = BeautifulSoup(html, 'lxml')  #'parsing HTML data'
        record=soup.findAll('article',{'data-automation':'normalJob'})
        for i in range(len(record)): #iteration through records
            s0=record[i].find('span',{'class':'lwHBT6d'}) #extract salary
            if s0:

                df_dict['Salary'].append(s0.text)
            else:
                df_dict['Salary'].append('NA')


            t0=record[i]['aria-label'] #job title
            if t0:
                df_dict['Title'].append(t0)
            else:
                df_dict['Title'].append('NA')

            c0=record[i].find('a',{'data-automation':"jobCompany"}) #company name
            if c0:
                df_dict['Company'].append(c0.text)
            else:
                df_dict['Company'].append('NA')

            d0=record[i].find('span',{'class':"bl7UwXp"}).text #short description
            if d0:
                df_dict['ShortDesp'].append(d0)
            else:
                df_dict['ShortDesp'].append('NA')

            l0=record[i].find('a',{'data-automation':"jobTitle"}) #link to details of full job description
            if l0:
                links='https://www.seek.com.au'+l0['href']
                rep=requests.get(links)
                #print(rep.status_code)
                if rep.status_code==200:
                    hsum=rep.text
                    soup_link=BeautifulSoup(hsum, 'lxml')
                    dsp=soup_link.find('div',{'data-automation':"mobileTemplate"})
                    if dsp:
                    #dsp.get_text()
                        df_dict['JobSummary'].append(dsp.get_text())

                        s1=soup_link.find('section',{'role':'region'})
                        s2=s1.findAll('strong')
                        df_dict['Location'].append(s2[0].text)
                        df_dict['Industry'].append(s2[1].text)
                    else:
                        df_dict['JobSummary'].append('NA')
                        df_dict['Location'].append('NA')
                        df_dict['Industry'].append('NA')
            
    return response.status_code

In [9]:
def category_scrape(filename,lk,nst,ned):
    index=0
    page=1
    html=[]
    df_dict={'Title':[],'Company':[],'Location':[], 'Salary':[],'JobSummary':[],'Industry':[],'ShortDesp':[]}
    for page in range(nst,ned+1):
        weblink=lk+str(page)
        print(weblink)
        r=scrape(weblink,df_dict)
        #print('return code:',r)
        if r!=200:
            break
    df=pd.DataFrame(df_dict)
    print(len(df))
    filename+='.csv'
    df.to_csv(filename, sep='\t', encoding='utf-8')